# CNN图像分类：从零实现到迁移学习

**目标**: 理解CNN架构，掌握迁移学习技术

**预计时间**: 90-120分钟

---

## 📋 本Notebook内容

1. 数据准备：CIFAR-10数据集
2. 从零实现简单CNN
3. 可视化卷积核和特征图
4. 使用预训练ResNet-50进行迁移学习
5. 对比从零训练vs迁移学习
6. 模型评估与可视化

In [ ]:
# 导入必要的库
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tqdm.notebook import tqdm

# 设置随机种子
torch.manual_seed(42)
np.random.seed(42)

# 检测设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'使用设备: {device}')

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

---

## 第1部分: 数据准备

CIFAR-10包含10个类别的60000张32×32彩色图像，每类6000张。

In [ ]:
# 数据增强
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

# 下载并加载数据集
trainset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transform_train
)

testset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transform_test
)

trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

# CIFAR-10类别
classes = ('飞机', '汽车', '鸟', '猫', '鹿', '狗', '青蛙', '马', '船', '卡车')

print(f'训练集大小: {len(trainset)}')
print(f'测试集大小: {len(testset)}')
print(f'类别数: {len(classes)}')

In [ ]:
# 可视化部分数据
def show_images(images, labels, preds=None, n=16):
    """显示图像网格"""
    fig, axes = plt.subplots(4, 4, figsize=(12, 12))
    axes = axes.ravel()
    
    for i in range(min(n, len(images))):
        # 反归一化
        img = images[i].cpu().numpy().transpose((1, 2, 0))
        mean = np.array([0.4914, 0.4822, 0.4465])
        std = np.array([0.2023, 0.1994, 0.2010])
        img = std * img + mean
        img = np.clip(img, 0, 1)
        
        axes[i].imshow(img)
        title = f'真实: {classes[labels[i]]}'
        if preds is not None:
            title += f'\n预测: {classes[preds[i]]}'
            color = 'green' if labels[i] == preds[i] else 'red'
            axes[i].set_title(title, color=color)
        else:
            axes[i].set_title(title)
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

# 显示一批训练数据
dataiter = iter(trainloader)
images, labels = next(dataiter)
show_images(images, labels, n=16)

---

## 第2部分: 从零实现CNN

构建一个简单的CNN架构：3个卷积层 + 2个全连接层

In [ ]:
class SimpleCNN(nn.Module):
    """简单的CNN架构"""
    
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()
        
        # 卷积层
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2, 2)
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(2, 2)
        
        # 全连接层
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 4 * 4, 256)
        self.relu4 = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, num_classes)
    
    def forward(self, x):
        # 卷积块1
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        
        # 卷积块2
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        
        # 卷积块3
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        x = self.pool3(x)
        
        # 全连接层
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu4(x)
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

# 创建模型
model_simple = SimpleCNN(num_classes=10).to(device)

# 打印模型结构
print(model_simple)

# 计算参数量
total_params = sum(p.numel() for p in model_simple.parameters())
trainable_params = sum(p.numel() for p in model_simple.parameters() if p.requires_grad)
print(f'\n总参数量: {total_params:,}')
print(f'可训练参数: {trainable_params:,}')

### 训练函数

In [ ]:
def train_model(model, trainloader, testloader, epochs=10, lr=0.001):
    """训练模型"""
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
    
    train_losses = []
    train_accs = []
    test_accs = []
    
    for epoch in range(epochs):
        # 训练阶段
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        pbar = tqdm(trainloader, desc=f'Epoch {epoch+1}/{epochs}')
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            pbar.set_postfix({
                'loss': running_loss / (pbar.n + 1),
                'acc': 100. * correct / total
            })
        
        train_loss = running_loss / len(trainloader)
        train_acc = 100. * correct / total
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        
        # 测试阶段
        model.eval()
        correct = 0
        total = 0
        
        with torch.no_grad():
            for images, labels in testloader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
        
        test_acc = 100. * correct / total
        test_accs.append(test_acc)
        
        print(f'Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Test Acc: {test_acc:.2f}%')
        
        scheduler.step()
    
    return train_losses, train_accs, test_accs

In [ ]:
# 训练简单CNN（建议使用GPU，CPU会很慢）
print("开始训练简单CNN...")
train_losses, train_accs, test_accs = train_model(
    model_simple,
    trainloader,
    testloader,
    epochs=10,
    lr=0.001
)

In [ ]:
# 绘制训练曲线
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(train_losses, label='训练损失')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('损失')
ax1.set_title('训练损失曲线')
ax1.legend()
ax1.grid(True)

ax2.plot(train_accs, label='训练准确率')
ax2.plot(test_accs, label='测试准确率')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('准确率 (%)')
ax2.set_title('准确率曲线')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

print(f'\n最终测试准确率: {test_accs[-1]:.2f}%')

---

## 第3部分: 可视化卷积核

查看第一层卷积核学到的滤波器

In [ ]:
def visualize_filters(model, layer_name='conv1', n_filters=32):
    """可视化卷积核"""
    # 获取第一层卷积核权重
    conv1_weights = model.conv1.weight.data.cpu()
    
    # 归一化到[0, 1]
    conv1_weights = (conv1_weights - conv1_weights.min()) / (conv1_weights.max() - conv1_weights.min())
    
    # 绘制卷积核
    fig, axes = plt.subplots(4, 8, figsize=(16, 8))
    axes = axes.ravel()
    
    for i in range(min(n_filters, conv1_weights.shape[0])):
        # 转换为可视化格式 (H, W, C)
        kernel = conv1_weights[i].permute(1, 2, 0).numpy()
        axes[i].imshow(kernel)
        axes[i].set_title(f'滤波器 {i+1}')
        axes[i].axis('off')
    
    plt.suptitle('第一层卷积核可视化', fontsize=16)
    plt.tight_layout()
    plt.show()

visualize_filters(model_simple)

### 可视化特征图

In [ ]:
def visualize_feature_maps(model, image, layer_names=['conv1', 'conv2', 'conv3']):
    """可视化特征图"""
    model.eval()
    
    # 注册钩子函数获取中间层输出
    activations = {}
    
    def get_activation(name):
        def hook(model, input, output):
            activations[name] = output.detach()
        return hook
    
    # 注册钩子
    hooks = []
    for name in layer_names:
        layer = getattr(model, name)
        hooks.append(layer.register_forward_hook(get_activation(name)))
    
    # 前向传播
    with torch.no_grad():
        _ = model(image.unsqueeze(0).to(device))
    
    # 移除钩子
    for hook in hooks:
        hook.remove()
    
    # 可视化
    fig, axes = plt.subplots(len(layer_names), 8, figsize=(16, len(layer_names)*2))
    
    for i, name in enumerate(layer_names):
        feature_map = activations[name][0].cpu()
        
        for j in range(8):
            if j < feature_map.shape[0]:
                axes[i, j].imshow(feature_map[j], cmap='viridis')
                axes[i, j].set_title(f'{name} - ch{j+1}')
            axes[i, j].axis('off')
    
    plt.suptitle('不同层的特征图可视化', fontsize=16)
    plt.tight_layout()
    plt.show()

# 选择一张图像
test_image, test_label = testset[0]
print(f'图像类别: {classes[test_label]}')

# 显示原图
img = test_image.numpy().transpose((1, 2, 0))
mean = np.array([0.4914, 0.4822, 0.4465])
std = np.array([0.2023, 0.1994, 0.2010])
img = std * img + mean
img = np.clip(img, 0, 1)

plt.figure(figsize=(4, 4))
plt.imshow(img)
plt.title(f'原图: {classes[test_label]}')
plt.axis('off')
plt.show()

# 可视化特征图
visualize_feature_maps(model_simple, test_image)

---

## 第4部分: 迁移学习 - ResNet-50

使用预训练的ResNet-50进行迁移学习

In [ ]:
# 加载预训练ResNet-50
model_resnet = torchvision.models.resnet50(pretrained=True)

# 冻结所有卷积层
for param in model_resnet.parameters():
    param.requires_grad = False

# 替换最后的全连接层
num_ftrs = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(num_ftrs, 10)

model_resnet = model_resnet.to(device)

# 打印可训练参数
trainable_params = sum(p.numel() for p in model_resnet.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model_resnet.parameters())
print(f'总参数量: {total_params:,}')
print(f'可训练参数: {trainable_params:,} ({100*trainable_params/total_params:.2f}%)')

In [ ]:
# 训练ResNet-50（只训练最后一层，会很快）
print("开始训练ResNet-50（迁移学习）...")
train_losses_resnet, train_accs_resnet, test_accs_resnet = train_model(
    model_resnet,
    trainloader,
    testloader,
    epochs=5,
    lr=0.001
)

---

## 第5部分: 性能对比

对比简单CNN vs ResNet-50迁移学习

In [ ]:
# 对比准确率
fig, ax = plt.subplots(figsize=(10, 6))

epochs_simple = list(range(1, len(test_accs) + 1))
epochs_resnet = list(range(1, len(test_accs_resnet) + 1))

ax.plot(epochs_simple, test_accs, 'o-', label='简单CNN（从零训练）', linewidth=2)
ax.plot(epochs_resnet, test_accs_resnet, 's-', label='ResNet-50（迁移学习）', linewidth=2)
ax.set_xlabel('Epoch', fontsize=12)
ax.set_ylabel('测试准确率 (%)', fontsize=12)
ax.set_title('模型性能对比', fontsize=14)
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)

plt.show()

print("\n=== 性能对比总结 ===")
print(f"简单CNN最终测试准确率: {test_accs[-1]:.2f}%")
print(f"ResNet-50迁移学习最终测试准确率: {test_accs_resnet[-1]:.2f}%")
print(f"准确率提升: {test_accs_resnet[-1] - test_accs[-1]:.2f}%")

---

## 第6部分: 模型评估

详细评估模型性能

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import itertools

def evaluate_model(model, testloader):
    """评估模型并生成混淆矩阵"""
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in testloader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.numpy())
    
    return np.array(all_labels), np.array(all_preds)

def plot_confusion_matrix(cm, classes, title='混淆矩阵'):
    """绘制混淆矩阵"""
    plt.figure(figsize=(10, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(title, fontsize=14)
    plt.colorbar()
    
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                ha="center", va="center",
                color="white" if cm[i, j] > thresh else "black")
    
    plt.ylabel('真实类别', fontsize=12)
    plt.xlabel('预测类别', fontsize=12)
    plt.tight_layout()
    plt.show()

# 评估ResNet-50
labels, preds = evaluate_model(model_resnet, testloader)
cm = confusion_matrix(labels, preds)
plot_confusion_matrix(cm, classes, title='ResNet-50迁移学习 - 混淆矩阵')

# 打印分类报告
print("\n=== 分类报告 ===")
print(classification_report(labels, preds, target_names=classes))

In [ ]:
# 可视化预测结果
dataiter = iter(testloader)
images, labels = next(dataiter)

model_resnet.eval()
with torch.no_grad():
    outputs = model_resnet(images.to(device))
    _, preds = outputs.max(1)

show_images(images, labels, preds.cpu(), n=16)

---

## 📝 总结

通过本Notebook，我们：

1. ✅ 从零实现了简单CNN，理解了卷积、池化、BN层的作用
2. ✅ 可视化了卷积核和特征图，直观理解CNN学到的特征
3. ✅ 使用预训练ResNet-50进行迁移学习
4. ✅ 对比了从零训练vs迁移学习的性能差异
5. ✅ 详细评估了模型性能（混淆矩阵、分类报告）

### 关键要点

- **CNN架构**: 卷积层提取特征，池化层降维，全连接层分类
- **迁移学习**: 在小数据集上，迁移学习显著优于从零训练
- **特征层次**: 浅层学习边缘/纹理，深层学习高级语义

### 下一步

- 🎯 尝试微调ResNet的后几层，观察性能提升
- 📚 学习其他经典架构（VGG、EfficientNet）
- 🚀 进入实战项目：[P01: 工业视觉检测](../../docs/stage4/projects/p01-industrial-vision/README.md)